In [ ]:
import dask
import dask.array

A = dask.array.random.random((1, 3000, 20000), chunks=1024)
B = dask.array.random.random((50, 1, 20000), chunks=1024)

C = A * B

dask.array.to_hdf5('myfile.hdf5', '/C', C)

In [ ]:
import numpy as np

In [ ]:
features_matrix.shape

In [ ]:
import numpy as np
import dask.array as da
import memory_profiler
%load_ext memory_profiler
%load_ext line_profiler


In [ ]:
Y = da.random.normal(size=(10000, 10000),
                     chunks=(1000, 1000))

In [ ]:
Y

In [ ]:
Y.shape, Y.size, Y.chunks


In [ ]:
mu = Y.mean(axis=0)
mu

In [ ]:
mu[0].compute()

In [1]:
import dask
import numpy as np
import dask.array as da
import h5py

import memory_profiler


In [2]:
def load_features_from_file(path):
    with open(path, 'r') as features_file:
        # Le as linhas
        # As linhas contem um feature set de uma musica
        # Pra cada set de uma musica, le como float cada feature
        # Retorna uma lista criada com esses floats
        #[[x1, x2, x3...]
        # [x1, x2, x3...]]
        return [[float(feature) for feature in feature_set.split()] for feature_set in features_file.readlines()]

In [7]:
features_matrix = np.matrix(load_features_from_file(
    '../features/all_features-music4all.txt'
    ))

In [16]:
lazy_feature_mat = da.from_array(features_matrix, chunks=(10000, 10000))
lazy_feature_mat

dask.array<array, shape=(109269, 120), dtype=float64, chunksize=(10000, 120), chunktype=numpy.matrix>

In [18]:
x = lazy_feature_mat.dot(lazy_feature_mat.T)
x

/home/giow/giow-tcc/venv/lib/python3.9/site-packages/dask/array/routines.py:255: PerformanceWarning: Increasing number of chunks by factor of 11
  intermediate = blockwise(


dask.array<tensordot, shape=(109269, 109269), dtype=float64, chunksize=(10000, 10000), chunktype=numpy.ndarray>

In [19]:
dask.array.to_hdf5('myfile.hdf5', '/C', x)

In [20]:
f = h5py.File('myfile.hdf5') # HDF5 file
d = f['/C']          # Pointer on on-disk array
d.shape                      # d can be very large


(109269, 109269)

In [21]:
x = da.from_array(d, chunks=(1000, 1000))
x

dask.array<array, shape=(109269, 109269), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>